In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from scipy import stats
from gensim.models import Word2Vec
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.utils import resample

In [2]:
df = pd.DataFrame()
df = pd.read_csv('data/amr-bank-struct-v1.6.csv', encoding = 'utf-8')
df.head()

,isfocus,focus,word,normalized_words,index,POS,-1POS,-2POS,+1POS,+2POS
0,1,chapter,Chapter,chapter,0,NN,XXXX,XXXX,CD,.
1,0,chapter,1,1,1,CD,NN,XXXX,.,XXXX
2,0,chapter,.,.,2,.,CD,NN,XXXX,XXXX
3,0,see,Once,once,0,RB,XXXX,XXXX,WRB,PRP
4,0,see,when,when,1,WRB,RB,XXXX,PRP,VBD


In [3]:
x = df.loc[:,['word','normalized_words','isfocus','index','POS','-1POS','-2POS','+1POS','+2POS']]
x.head()

,word,normalized_words,isfocus,index,POS,-1POS,-2POS,+1POS,+2POS
0,Chapter,chapter,1,0,NN,XXXX,XXXX,CD,.
1,1,1,0,1,CD,NN,XXXX,.,XXXX
2,.,.,0,2,.,CD,NN,XXXX,XXXX
3,Once,once,0,0,RB,XXXX,XXXX,WRB,PRP
4,when,when,0,1,WRB,RB,XXXX,PRP,VBD


In [4]:
# x['normalized_words'] = x['normalized_words']+' '+x['index'].astype(str)+' '+x['POS']+' '+x['-1POS']+' '+x['-1POS']+' '+x['-2POS']+' '+x['+1POS']+' '+x['+2POS']
x['normalized_words'] = x['word']+' '+x['normalized_words']+' '+x['index'].astype(str)+' '+x['POS']+' '+x['-1POS']+' '+x['-1POS']+' '+x['-2POS']+' '+x['+1POS']+' '+x['+2POS']
x = x.iloc[0:20000,0:3]

In [5]:
# df_0 = x[df.isfocus == 0]
# df_1 = x[df.isfocus == 1]

# df_majority_downsampled = resample(df_0, 
#                                  replace=False,
#                                  n_samples=df_1.size,     
#                                  random_state=123) 

# df_downsampled = pd.concat([df_majority_downsampled, df_1])

# X = df_downsampled['normalized_words'].values
# y = df_downsampled['isfocus'].values
X = x['normalized_words'].values
y = x['isfocus'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5,
                                                    random_state = 1234,stratify= y)

In [6]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
tokenizer_obj = Tokenizer()
total_words = X_train + X_test
tokenizer_obj.fit_on_texts(total_words)

max_length = max([len(s.split()) for s in total_words])

vocab_size =len(tokenizer_obj.word_index)+1

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding = 'post')
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length, padding ='post')

In [7]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,GRU
from keras.models import Sequential
from keras.layers.embeddings import Embedding

EMBEDDING_DIM =100

model=Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length = max_length))
model.add(GRU(units=32, dropout =0.5, recurrent_dropout=0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 17, 100)           558700    
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 571,501
Trainable params: 571,501
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.fit(X_train_pad, y_train, batch_size=128, epochs =50, validation_data= (X_test_pad, 
                                                                            y_test),verbose=2)


Train on 10000 samples, validate on 10000 samples
Epoch 1/50
 - 4s - loss: 0.3159 - acc: 0.9300 - val_loss: 0.1999 - val_acc: 0.9492
Epoch 2/50
 - 3s - loss: 0.2043 - acc: 0.9493 - val_loss: 0.1991 - val_acc: 0.9492
Epoch 3/50
 - 3s - loss: 0.1992 - acc: 0.9493 - val_loss: 0.1905 - val_acc: 0.9492
Epoch 4/50
 - 3s - loss: 0.1656 - acc: 0.9503 - val_loss: 0.1342 - val_acc: 0.9564
Epoch 5/50
 - 3s - loss: 0.1251 - acc: 0.9574 - val_loss: 0.1181 - val_acc: 0.9638
Epoch 6/50
 - 3s - loss: 0.1011 - acc: 0.9616 - val_loss: 0.1234 - val_acc: 0.9623
Epoch 7/50
 - 3s - loss: 0.0936 - acc: 0.9661 - val_loss: 0.1253 - val_acc: 0.9607
Epoch 8/50
 - 3s - loss: 0.0850 - acc: 0.9696 - val_loss: 0.1314 - val_acc: 0.9588
Epoch 9/50
 - 3s - loss: 0.0782 - acc: 0.9711 - val_loss: 0.1355 - val_acc: 0.9567
Epoch 10/50
 - 3s - loss: 0.0790 - acc: 0.9695 - val_loss: 0.1348 - val_acc: 0.9592
Epoch 11/50
 - 3s - loss: 0.0757 - acc: 0.9727 - val_loss: 0.1366 - val_acc: 0.9586
Epoch 12/50
 - 3s - loss: 0.0742 - 

In [11]:
y_pred = model.predict(x = X_test_pad)
for i in range(len(y_pred)):
    if y_pred[i][0] >= 0.9:
        y_pred[i][0] = 1
    else:
        y_pred[i][0] = 0

In [12]:
from sklearn.metrics import f1_score,precision_score,recall_score
print ('f1_score: ',f1_score(y_true=y_test, y_pred=y_pred))
print ('precision_score:',precision_score(y_true=y_test, y_pred=y_pred))
print ('recall_score:',recall_score(y_true=y_test, y_pred=y_pred))
cm_svm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print (cm_svm)
def accuracy(cm):
    return np.trace(cm)/np.sum(cm)
print (accuracy(cm_svm))


f1_score:  0.43454038997214495
precision_score: 0.7428571428571429
recall_score: 0.30708661417322836
[[9438   54]
 [ 352  156]]
0.9594


In [730]:
from keras.utils.vis_utils import plot_model
# test_sample_1 = 'say 9 VBD NN NN DT , ``'
# test_sample_2 = 'prince 9 NN JJ JJ DT , CC'
# test_sample_3 = 'reveal 26 VBN VBD VBD NN TO PRP'
# test_sample_4 = 'chapter 0 NN XXXX XXXX XXXX CD .'
# test_sample_5 = 'little 21 JJ DT DT IN NN POS'
# test_sample_6 = 'i 1 PRP `` XXXX VBP RB'

test_sample_1 = 'see 6 VBN VBN VBN RB IN DT'
test_sample_2 = 'present 8 VBN VBD VBD NN PRP TO'
test_sample_3 = 'account 24 NN IN IN VBZ IN DT'
test_sample_4 = 'he 1 PRP CC CC XXXX VBD IN'
test_sample_5 = 'in 3 IN VBD VBD PRP JJ NN'
test_sample_6 = 'of 23 IN NN NN IN NN ,'
test_samples=[test_sample_1,test_sample_2,test_sample_3,test_sample_4,test_sample_5,test_sample_6]
test_samples_tokens = tokenizer_obj.texts_to_sequences (test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=max_length)

model.predict(x=test_samples_tokens_pad)


    



array([[0.00641237],
       [0.0059375 ],
       [0.00600799],
       [0.0052671 ],
       [0.00553304],
       [0.00549879]], dtype=float32)

SyntaxError: invalid syntax (<ipython-input-299-16b4a43ecc59>, line 5)